In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-08 03:37:35,108 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-08 03:37:35,110 - settings - Setting database.user to celiib
INFO - 2021-01-08 03:37:35,110 - settings - Setting database.password to newceliipass
INFO - 2021-01-08 03:37:35,119 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-08 03:37:35,120 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-08 03:37:35,131 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-08 03:37:35,368 - settings - Setting enable_python_native_blobs to True


In [2]:
test_mode = False

In [3]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-08 03:37:35,401 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-08 03:37:35,403 - settings - Setting database.user to celiib
INFO - 2021-01-08 03:37:35,404 - settings - Setting database.password to newceliipass
INFO - 2021-01-08 03:37:35,408 - settings - Setting enable_python_native_blobs to True


Sleeping 117 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-08 03:37:35,653 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [4]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [5]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4076


In [6]:
#so that it will have the adapter defined
from datajoint_utils import *

In [7]:
decimation_version = 0
decimation_ratio = 0.25
key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134947393276,0,0.25
864691135012398582,0,0.25
864691135012571126,0,0.25
864691135012905110,0,0.25
864691135065018692,0,0.25
864691135065024068,0,0.25
864691135065032772,0,0.25
864691135065033540,0,0.25
864691135081745143,0,0.25
864691135081756919,0,0.25


In [8]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = (minnie.Decimation().proj(decimation_version='version')  & 
                  dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
                  & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()) 

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
        
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [9]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,00471f14c125b842baffaf6ebff1eed1,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.129,at-node2,29,25193,2021-01-07 20:31:24
__decomposition,12dc98a8b9a21eadf99f39bdbf6efd06,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.165,at-node38,30,25188,2021-01-07 20:10:43
__decomposition,199727e7563f1791c7d099384821361b,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.156,at-node29,29,25158,2021-01-07 20:58:34
__decomposition,223292d754d01c10ec97cae17b086998,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.155,at-node28,29,25175,2021-01-07 20:16:19
__decomposition,3494d03ffcd00736756886336ff511f2,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.170,at-node43,29,25209,2021-01-07 21:06:52
__decomposition,3c6b779c3fa3f7a2d5e6ae98dd02bdf1,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.147,at-node20,29,25176,2021-01-07 20:13:15
__decomposition,42e75b2831b8230d70728bb3dd0d5385,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.166,at-node39,29,25180,2021-01-07 21:00:59
__decomposition,53dbe0a55f1b2433ab9972a890d4dcac,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.161,at-node34,29,25186,2021-01-07 21:06:29
__decomposition,86d93ed55e0b6413ecba6b597ce7eab3,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.158,at-node31,29,25198,2021-01-07 20:15:24
__decomposition,89c0286c7575dad401ed45a9f2b32404,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.138,at-node11,29,25170,2021-01-07 21:20:23


In [10]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-01-08 03:37:35,840 - autopopulate - Found 1 keys to populate
INFO - 2021-01-08 03:37:35,849 - connection - Transaction started
INFO - 2021-01-08 03:37:35,851 - autopopulate - Populating: {'segment_id': 864691135589917707, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started


----- Working on 864691135589917707-------
somas = [[<trimesh.Trimesh(vertices.shape=(15879, 3), faces.shape=(31462, 3))>, <trimesh.Trimesh(vertices.shape=(14626, 3), faces.shape=(26566, 3))>, <trimesh.Trimesh(vertices.shape=(13244, 3), faces.shape=(26342, 3))>], array([605.6347, 605.6347, 605.6347]), array([0.81 , 0.552, 0.841])]

--- Beginning preprocessing of 864691135589917707---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(15879, 3), faces.shape=(31462, 3))>, <trimesh.Trimesh(vertices.shape=(14626, 3), faces.shape=(26566, 3))>, <trimesh.Trimesh(vertices.shape=(13244, 3), faces.shape=(26342, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(15879, 3), faces.shape=(31462, 3))>, <trimesh.Trimesh(vertices.shape=(14626, 3), faces.shape=(26566, 3))>, <trimesh.Trimesh(vertices.

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:09<00:00,  9.77s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 12.318209648132324
connecting at the root
branches_touching_root = [79]
length of Graph = 26362
Working on path [16276. 16383. 16473. 16533. 16580. 16630. 16647.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [17724. 17729. 17733. 17735. 17741.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [23469. 23493. 23518. 23542. 23550.]
path_degrees = [3, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 79, len(kept_branches_idx) = 77
empty_indices % = 0.0
 conflict_indices % = 0.008799377450150428


...


AFTER face_lookup_resolved_test



main_branch = 78
Status of Main limb stitch point moved = True
Status of Main limb stitch point moved = True
Total time for stitching floating pieces = 13.460984706878662
Total time for Concept Networks = 37.90754222869873
Total time for all mesh and skeletonization decomp = 1263.1358804702759
No soma volume ratios so computing them now
Using Poisson Surface Reconstruction for watertightness in soma_volume_ratio
xvfb-run -n 4076 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_774000.off -o /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_774000_poisson.off -s /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/poisson_374162.mls
removed temporary input file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_774000.off
removed temporary output file: /notebooks/Auto_Proofreading/Preprocessing_Pipeline/Poisson_temp/neuron_774000_poisson.off
mesh.is_watertight =

Working on limb L0 branch 107
Working on limb L0 branch 18
Working on limb L0 branch 27
Working on limb L0 branch 32
Working on limb L0 branch 33
Working on limb L0 branch 102
Working on limb L0 branch 53
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 57
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 58
Working on limb L0 branch 85
Working on limb L0 branch 97
Working on limb L0 branch 72
Working on limb L0 branch 77
Working on limb L0 branch 60
Working on limb L0 branch 65
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 69
Working on limb L0 branch 79
Working on limb L0 branch 105
Working on limb L0 branch 42
Working on limb L0 branch 59
Working on limb L0 branch 109
Assigning the old width calculation because no valid new widths
Working on limb L0 branch 63
Working on limb L0 branch 68
Working on limb L0 branch 20
Working on limb L0 branch 61
Working on limb


Working on limb L0 branch 55
Working on limb L0 branch 43
No spines and using precomputed width
Working on limb L0 branch 47
No spines and using precomputed width
Working on limb L0 branch 13
Working on limb L0 branch 36
Working on limb L0 branch 48
No spines and using precomputed width
Working on limb L0 branch 50
Working on limb L0 branch 6
Working on limb L0 branch 17
Working on limb L0 branch 10
Working on limb L0 branch 54
No spines and using precomputed width
Working on limb L0 branch 70
Working on limb L0 branch 71
Working on limb L0 branch 4
Working on limb L0 branch 15
Working on limb L0 branch 35
Working on limb L0 branch 56
Working on limb L0 branch 91
Working on limb L0 branch 93
Working on limb L0 branch 94
Working on limb L0 branch 108
No spines and using precomputed width
Working on limb L0 branch 0
Working on limb L0 branch 3
Working on limb L0 branch 28
Working on limb L0 branch 31
No spines and using precomputed width
Working on limb L0 branch 62
No spines and using 

Working on limb L3 branch 13
Working on limb L3 branch 14
Working on limb L3 branch 15
Working on limb L3 branch 0
No spines and using precomputed width
Working on limb L3 branch 3
No spines and using precomputed width
Working on limb L3 branch 12
Working on limb L3 branch 16
Working on limb L4 branch 0
No spines and using precomputed width
Working on limb L4 branch 1
Working on limb L4 branch 2
No spines and using precomputed width
Working on limb L4 branch 3
Working on limb L4 branch 4
Working on limb L4 branch 7
Working on limb L4 branch 8
Working on limb L4 branch 5
Working on limb L4 branch 6
Working on limb L10 branch 1
Working on limb L10 branch 0
No spines and using precomputed width
Working on limb L10 branch 2
No spines and using precomputed width
Working on limb L5 branch 11
No spines and using precomputed width
Working on limb L5 branch 13
No spines and using precomputed width
Working on limb L5 branch 16
No spines and using precomputed width
Working on limb L5 branch 18
No